# Computer Theory Project - Phase 2 (CFG to CNF)

`Members`
1. Antony Kiroles 202000897
2. Donia Khaled 202001874
3. Eslam Ahmed 202000039
3. Mariam Barakat 202000210

In [277]:
import copy
import re

In [278]:
# 1- Create new start and add it to the inputs

#test = ['S->ABA','A->aA|ε','B->bBc|ε']
test = ['S->ASB|a', 'A->aAS|a|ε', 'B->SbS|A|bb']

newStart = 'N->'+test[0][0]

newTest = [None] * (len(test) + 1)

newTest[0] = newStart

newTest[1:] = test

print(newTest)

['N->S', 'S->ASB|a', 'A->aAS|a|ε', 'B->SbS|A|bb']


In [279]:
def addAndSymbol(input:list[str])->list[str]:
    operators = ['|','>','-','.']
    for rule in range(len(input)):
        rules = list(input[rule])
        insertOffset = 0
        for i in range(1, len(input[rule])):
            if input[rule][i] not in operators and input[rule][i-1] not in operators:
                a = rules[i]
                b = rules[i-1]
                rules.insert(i+insertOffset,'.')
                insertOffset += 1
        input[rule] = ''.join(rules)
    return input

def stringListToCFG(input:list[str])->dict[str,set[str]]:
    input = addAndSymbol(input)
    input = [x.split('->') for x in input]
    input = [{x[0]:set(x[1].split('|'))} for x in input]
    return input
print(addAndSymbol(newTest))
print(stringListToCFG(newTest))

['N->S', 'S->A.S.B|a', 'A->a.A.S|a|ε', 'B->S.b.S|A|b.b']
[{'N': {'S'}}, {'S': {'A.S.B', 'a'}}, {'A': {'ε', 'a.A.S', 'a'}}, {'B': {'S.b.S', 'b.b', 'A'}}]


In [280]:
from itertools import combinations

# helper functions that outputs the combinations of all the letters that need to be removed
def remove_indices(string_,to_remove):
    # find position all what you need to remove
    positions_toremove =[] 
    for j in range(len(string_)):
        if string_[j] == to_remove:
            positions_toremove.append(j) 
    # generate combinations of this index
    indecies=[]
    for j in range(1,len(positions_toremove)+1):
        indecies.extend(list(combinations(positions_toremove, r=j)))

    return indecies   

In [281]:
# a helper function to loop through the indices to remove and remove them 
def generate_combinations(string_,indecies):
    # loop through string and remove according to the combination
    temp_str=set()
    c=list(string_)
    for i in indecies:
        x=c.copy()
        for j in i:
            x[j]=''

        added = ''.join(x)
        if len(added) == 0:
            temp_str.add('ε') 
        else:
            temp_str.add(added) 

    return temp_str   

In [282]:
# a helper function to make sure that there is no null to further remove
def no_null(CFG, start):
    for key, values in CFG.items():
        if "ε" in values and key != start:
            return False
    return True

In [283]:
# ========================== remove null productions 
def remove_null_production(CFG, start):
    ###finished = []
    while no_null(CFG, start) == False:
        for key, values in CFG.items():
            # start by finding any epsilon to be able to remove and replace it 
            ### if key in finished and "ε" in values and key != start:
            ###     # remove the epsilon and call it a day
            ###     CFG[key].remove('ε')
            
            if "ε" in values and key != start:
                # find all the values of the key 
                CFG[key].remove('ε')
                for keys2, values2 in CFG.items():
                    # loop to find the key that had the epsolin 
                    for j in values2: 
                        if key in ''.join(list(j)):   
                            #search for the key in the values 
                            CFG[keys2] = CFG[keys2].union(generate_combinations(j,remove_indices(j,key)))
                ###finished.append(key)
                CFGcopy = copy.deepcopy(CFG)
                for i in CFGcopy:
                    for j in CFGcopy[i]:
                        # noEps = re.search('[a-zA-Z]+', j)
                        CFG[i].add(j.replace('..', '.'))
                        if (j != j.replace('..', '.')):
                            CFG[i].remove(j)
                            j = j.replace('..', '.')
                        if (j[0] == '.' or j[-1] == '.'):
                            CFG[i].add(j.strip('.'))
                            CFG[i].remove(j)
                            
    return CFG

In [284]:
CFG = stringListToCFG(newTest)
CFG 

[{'N': {'S'}},
 {'S': {'A.S.B', 'a'}},
 {'A': {'a', 'a.A.S', 'ε'}},
 {'B': {'A', 'S.b.S', 'b.b'}}]

In [285]:
CFGdict = {}
for i in range (0, len(CFG)):
    CFGdict.update(CFG[i])
    
CFGdict

{'N': {'S'},
 'S': {'A.S.B', 'a'},
 'A': {'a', 'a.A.S', 'ε'},
 'B': {'A', 'S.b.S', 'b.b'}}

In [286]:
CFG = remove_null_production(CFGdict, newStart[0])
CFG

{'N': {'S'},
 'S': {'A.S', 'A.S.B', 'S', 'S.B', 'a'},
 'A': {'a', 'a.A.S', 'a.S'},
 'B': {'A', 'S.b.S', 'b.b'}}

In [287]:
# a helper function to search for all the nonterminals that are alone and adds them to a list
def contain_nonterminal(set_value):
    set_of_nonterminals = set()
    for i in set_value:
        if len(i) == 1 and i.isupper():
            set_of_nonterminals.add(i)
    return set_of_nonterminals


In [288]:
# a helper function to make sure that there is no nontermianls alone anymore 
def done(CFG):
    for k, v in CFG.items():
        if len(contain_nonterminal(v)) != 0:
            return False
    return True

In [289]:
# =============================== Unit production final function
def remove_unit_production(CFG):

    while done(CFG) != True:
        # loop through the dictionary to find nontemrminals that are alone
        for primary_key, set_value in CFG.items():
            set_non_terminals = contain_nonterminal(set_value)
            if len(set_non_terminals) != 0:
                #in case there is multiple nonterminals alone in a value
                for i in set_non_terminals:
                    # delete all the non-terminals
                    CFG[primary_key].remove(i)
                    # # union of the primary values and they nonterminal values we searched for 
                    CFG[primary_key] =  CFG[primary_key].union(CFG[i])
                    # delete if the primary key is equal to a nonterminal
                    if i == primary_key and i in CFG[primary_key]:
                        CFG[primary_key].remove(i)
    return CFG

In [290]:
#CFG2 = {"S": {"A.a", "B"}, "A": {"a", "b.c", "B"}, "B": {"A", "b.b"}}
CFG = remove_unit_production(CFG)
print(CFG)

{'N': {'A.S.B', 'A.S', 'a', 'S.B'}, 'S': {'A.S.B', 'A.S', 'S.B', 'a'}, 'A': {'a.S', 'a.A.S', 'a'}, 'B': {'b.b', 'a.A.S', 'a', 'S.b.S', 'a.S'}}


In [291]:
# test = {'S': {'a.S', 'A', 'C'}, 'A': {'a'}, 'B': {'a.a'}, 'C': {'a.C.b'}}
# start = 'S'

def eliminateUselessProd(cfg, start):
    testCopy = copy.deepcopy(cfg)
    
    # non-generating
    non_generating = set()
    for i in cfg[start]:
        for j in i:
            # case1: if the start state has a non-terminal that is not a key, meaning it does not have transitions, it is added to the non-generating set
            if (j not in cfg.keys() and j.isupper()): 
                non_generating.add(j)
                continue
            # case2: if a non-terminal transitions to a terminal AND nonterminal, without and epsilon, meaning the nonterminal cannot be removed, 
            # it is considered as non generating. Checks if its uppercase and does not have an epsilon transition and any transitions to terminal
            if ((j.isupper()) and ('' not in cfg[j]) and (len(cfg[j]) == 1)):
                for k in str(cfg[j]):
                    if k.isupper():
                        non_generating.add(j)
    
    # deletes the element that has a non-generating non-terminal
    for i in testCopy[start]:
        for j in i:
            if ((j in non_generating)):    
                cfg[start].remove(i)
    # deletes the non-generating non-terminal from the cfg 
    for i in non_generating:
        del cfg[i]
                        
    # Non-reachable
    reachable = {}
    non_reachable = {}
    
    for i in cfg[start]:
        for j in i:         # checks the non-terminals in the start state and appends them to reachable dict with their values
            if (j.isupper()):
                reachable[j] = cfg[j]   
            
    for i in cfg.keys():   # appends any non-terminals found in the CFG thats not in the reachable dict and appends it to the non reachable dict
        if ((i not in reachable.keys()) and i != start):
            non_reachable[i] = cfg[i]
        
    for i in testCopy:     # goes over the non-terminals that cannot be reached from the start state and deletes them
        if i in non_reachable:
            del cfg[i]
            
    return cfg

CFG = eliminateUselessProd(CFG, newStart[0])

    

In [292]:
print(CFG)

{'N': {'A.S.B', 'A.S', 'a', 'S.B'}, 'S': {'A.S.B', 'A.S', 'S.B', 'a'}, 'A': {'a.S', 'a.A.S', 'a'}, 'B': {'b.b', 'a.A.S', 'a', 'S.b.S', 'a.S'}}


In [293]:
def ReduceStringToEvens(rule:str,originalInput:dict[str,set[str]],currentDict:dict[str,set[str]],currentCounter:int)->tuple[str,dict[str,str],int]:
    output = {}
    if rule.count('.') == 1:
        return rule,{},currentCounter
    splitString = rule.split('.')
    if(len(splitString)%2==1):
        evenRule = None
        firstTwoLiterals = rule[:rule.index('.',rule.index('.')+1)]
        if {firstTwoLiterals} in originalInput.values():
            newRuleKey = list(originalInput.keys())[list(originalInput.values()).index({firstTwoLiterals})]
            evenRule = rule.replace(firstTwoLiterals,newRuleKey)
        elif {firstTwoLiterals} not in currentDict.values():
            newRuleKey = f'K{currentCounter}'
            currentCounter += 1
            newRuleValue= firstTwoLiterals
            output[newRuleKey] = {newRuleValue}
            evenRule = rule.replace(firstTwoLiterals,newRuleKey)
        else: 
            newRuleKey = list(currentDict.keys())[list(currentDict.values()).index({firstTwoLiterals})]
            evenRule = rule.replace(firstTwoLiterals,newRuleKey)
    else:
        evenRule = rule

    currentDict.update(output)
    if len(evenRule.split('.'))>2:
        left = '.'.join(splitString[:len(splitString)//2])
        right = '.'.join(splitString[len(splitString)//2:])
        leftSet = {left}
        if leftSet in originalInput.values():
            leftKey = list(originalInput.keys())[list(originalInput.values()).index(leftSet)]
        elif leftSet in output.values():
            leftKey = list(output.keys())[list(output.values()).index(leftSet)]
        elif leftSet not in currentDict.values():
            leftKey = currentCounter
            currentCounter += 1
            leftKey = f'K{leftKey}'
        else:
            leftKey = list(currentDict.keys())[list(currentDict.values()).index({left})]
        splitStringLeft = ReduceStringToEvens(left,originalInput,currentDict,currentCounter)
        currentCounter = splitStringLeft[2]
        output.update(splitStringLeft[1])
        output[f'{leftKey}'] = {splitStringLeft[0]}

        rightSet = {right}
        if rightSet in originalInput.values():
            rightKey = list(originalInput.keys())[list(originalInput.values()).index(rightSet)]
        elif rightSet in output.values():
            rightKey = list(output.keys())[list(output.values()).index(rightSet)]
        elif rightSet not in currentDict.values():
            rightKey = currentCounter
            currentCounter += 1
            rightKey = f'K{rightKey}'
        else:
            rightKey = list(currentDict.keys())[list(currentDict.values()).index(rightSet)]
        splitStringRight = ReduceStringToEvens(right,originalInput,currentDict,currentCounter)
        output[f'{rightKey}'] = {splitStringRight[0]}
        currentCounter = splitStringRight[2]
        output.update(splitStringRight[1])

        evenRule = f'{leftKey}.{rightKey}'
    
    return evenRule,output,currentCounter 
    
    
def ToCNF(input : dict[str,set[str]])->dict[str,set[str]]:
    output = {}
    newRuleCounter = 0
    for k,v in input.items():
        FinalValue = set()
        for rule in v:
            if(len(rule)>1):
                elements = rule.split('.')
                for i in range(len(elements)):
                    if(elements[i].islower()):
                        if {elements[i]} in input.values():
                            newRuleKey = list(input.keys())[list(input.values()).index({elements[i]})]
                            elements[i] = newRuleKey
                        elif {elements[i]} not in output.values():
                            newRuleKey = f'K{newRuleCounter}'
                            newRuleCounter += 1
                            newRuleValue = elements[i]
                            output[newRuleKey] = {newRuleValue}
                            elements[i] = newRuleKey
                        else: 
                            newRuleKey = list(output.keys())[list(output.values()).index({elements[i]})]
                            elements[i] = newRuleKey
                rule = '.'.join(elements)
            if len(rule.split('.'))<=2:
                FinalValue.add(rule)
                continue
            final = ReduceStringToEvens(rule,input,output,newRuleCounter)
            newRuleCounter = final[2]
            output.update(final[1])
            evenRule = final[0]
            FinalValue.add(evenRule)
        output[k] = FinalValue
    sortedOutput = {}
    for k in input.keys():
        sortedOutput[k] = output[k]
    sortedOutput.update(output)
    return sortedOutput

print(CFG)
ToCNF(CFG)

{'N': {'A.S.B', 'A.S', 'a', 'S.B'}, 'S': {'A.S.B', 'A.S', 'S.B', 'a'}, 'A': {'a.S', 'a.A.S', 'a'}, 'B': {'b.b', 'a.A.S', 'a', 'S.b.S', 'a.S'}}


{'N': {'A.S', 'K0.B', 'S.B', 'a'},
 'S': {'A.S', 'K0.B', 'S.B', 'a'},
 'A': {'K1.S', 'K2.S', 'a'},
 'B': {'K1.S', 'K2.S', 'K3.K3', 'K4.S', 'a'},
 'K0': {'A.S'},
 'K1': {'a'},
 'K2': {'K1.A'},
 'K3': {'b'},
 'K4': {'S.K3'}}